In [0]:
!pip install scikit-learn
!pip install numpy==1.16.1

In [0]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/')
!ls
os.chdir('My Drive/Computer_Vision_Masters/Wearable_Sensors_Code/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
'My Drive'


In [0]:
## Libraries
import numpy as np
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics.classification import accuracy_score, recall_score, f1_score
import scipy.stats as st
import sys
import warnings
warnings.filterwarnings('ignore')
np.random.seed(12227)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [0]:
def DataPreparation(data_input_file):
  print('Proposed Model {}'.format(data_input_file))
  data = np.load(data_input_file)
  X = data['X']
  X = X[:, 0, :, :]
  Y = data['y']
  folds = data['folds']
  classes_number = Y.shape[1]
  print(classes_number)
  Y = np.argmax(Y, axis=1)
  return X,Y,folds,classes_number

In [0]:
def A(sample):
    feat = []
    for col in range(0,sample.shape[1]):
        average = np.average(sample[:,col])
        feat.append(average)

    return feat

def SD(sample):
    feat = []
    for col in range(0, sample.shape[1]):
        std = np.std(sample[:, col])
        feat.append(std)

    return feat

def AAD(sample):
    feat = []
    for col in range(0, sample.shape[1]):
        data = sample[:, col]
        add = np.mean(np.absolute(data - np.mean(data)))
        feat.append(add)

    return feat

def ARA(sample):
    #Average Resultant Acceleration[1]:
    # Average of the square roots of the sum of the values of each axis squared √(xi^2 + yi^2+ zi^2) over the ED
    feat = []
    sum_square = 0
    sample = np.power(sample, 2)
    for col in range(0, sample.shape[1]):
        sum_square = sum_square + sample[:, col]

    sample = np.sqrt(sum_square)
    average = np.average(sample)
    feat.append(average)
    return feat

def TBP(sample):
    from scipy import signal
    feat = []
    sum_of_time = 0
    for col in range(0, sample.shape[1]):
        data = sample[:, col]
        peaks = signal.find_peaks_cwt(data, np.arange(1,4))

        feat.append(peaks)

    return feat

In [0]:
## Feature Extraction

def feature_extraction(X):
    # Extracts the features, as mentioned by Catal et al. 2015
    # Average - A,
    # Standard Deviation - SD,
    # Average Absolute Difference - AAD,
    # Average Resultant Acceleration - ARA(1),
    # Time Between Peaks - TBP
    X_tmp = []
    for sample in X:
        features = A(sample)
        features = np.hstack((features, A(sample)))
        features = np.hstack((features, SD(sample)))
        features = np.hstack((features, AAD(sample)))
        features = np.hstack((features, ARA(sample)))
        #features = np.hstack((features, TBP(sample)))
        X_tmp.append(features)

    X = np.array(X_tmp)
    return X

In [0]:
import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.2.0-rc2
/device:GPU:0
Num GPUs Available:  1


In [0]:
def Model(num_classes):
  layers = [
      tf.keras.layers.Dense(128, activation = tf.nn.relu),
      tf.keras.layers.Dense(64, activation = tf.nn.relu),
      tf.keras.layers.Dense(32, activation = tf.nn.relu),
      tf.keras.layers.Dense(num_classes, activation = 'softmax')
  ]
  model = tf.keras.Sequential(layers)
  return model

In [0]:
from tensorflow.keras.callbacks import EarlyStopping
def TrainAndReportAccuracy(X,Y,folds,num_classes):
  avg_acc = []
  train_idx = folds[0][0]
  test_idx = folds[0][1]
  X_train = X[train_idx]
  X_test = X[test_idx]
  Total_Ytrain = Y[train_idx]
  Total_Ytest = Y[test_idx]
  Total_Xtrain = feature_extraction(X_train)
  Total_Xtest = feature_extraction(X_test)
  from sklearn.preprocessing import StandardScaler
  from sklearn.decomposition import PCA
  for i in range(1, len(folds)):
          train_idx = folds[i][0]
          test_idx = folds[i][1]

          X_train = X[train_idx]
          X_test = X[test_idx]
          Y_train = Y[train_idx]
          Y_test = Y[test_idx]
          X_train = feature_extraction(X_train)
          X_test = feature_extraction(X_test)
          Total_Xtrain=np.append(Total_Xtrain, X_train,axis=0)
          Total_Ytrain=np.append(Total_Ytrain, Y_train)
          Total_Xtest=np.append(Total_Xtest, X_test,axis=0)
          Total_Ytest=np.append(Total_Ytest, Y_test)


  scaler = StandardScaler()
  scaler.fit(Total_Xtrain)
  Total_Xtrain = scaler.transform(Total_Xtrain)
  Total_Xtest = scaler.transform(Total_Xtest)
  pca = PCA(.95)
  pca.fit(Total_Xtrain)
  Total_Xtrain = pca.transform(Total_Xtrain)
  Total_Xtest = pca.transform(Total_Xtest)
  print(Total_Xtrain.shape, Total_Ytrain.shape)
  print(Total_Xtest.shape,Total_Ytest.shape)

  model = Model(num_classes)
  es = EarlyStopping(monitor='sparse_categorical_accuracy', mode='max', verbose=1, patience=15)
  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=[tf.keras.metrics.sparse_categorical_accuracy])
  history = model.fit(Total_Xtrain, Total_Ytrain, batch_size=32, epochs=200,callbacks=[es])
  _,acc=model.evaluate(Total_Xtest,Total_Ytest)
  avg_acc.append(acc)
  print('________________________________________________________________')
  print('Mean Accuracy[{:.4f}] '.format(np.mean(avg_acc)))
  

In [0]:
def Run(data_input_file):
  X,Y,folds,num_classes=DataPreparation(data_input_file)
  TrainAndReportAccuracy(X,Y,folds,num_classes)

In [0]:
Run('data/LOSO/MHEALTH.npz')

Proposed Model data/LOSO/MHEALTH.npz
12
(22995, 93) (22995,)
(2555, 93) (2555,)
Epoch 1/100
719/719 [==============================] - 2s 3ms/step - loss: 0.2638 - sparse_categorical_accuracy: 0.9417
Epoch 2/100
719/719 [==============================] - 2s 3ms/step - loss: 0.0209 - sparse_categorical_accuracy: 0.9927
Epoch 3/100
719/719 [==============================] - 2s 3ms/step - loss: 0.0114 - sparse_categorical_accuracy: 0.9955
Epoch 4/100
719/719 [==============================] - 2s 3ms/step - loss: 0.0023 - sparse_categorical_accuracy: 0.9994
Epoch 5/100
719/719 [==============================] - 2s 3ms/step - loss: 0.0163 - sparse_categorical_accuracy: 0.9949
Epoch 6/100
719/719 [==============================] - 2s 3ms/step - loss: 3.5715e-04 - sparse_categorical_accuracy: 1.0000
Epoch 7/100
719/719 [==============================] - 2s 3ms/step - loss: 0.0066 - sparse_categorical_accuracy: 0.9978
Epoch 8/100
719/719 [==============================] - 2s 3ms/step - loss: 9

In [0]:
Run('data/LOTO/MHEALTH.npz')

Proposed Model data/LOTO/MHEALTH.npz
12
(22995, 22) (22995,)
(2555, 22) (2555,)
Epoch 1/200
719/719 [==============================] - 2s 3ms/step - loss: 0.1531 - sparse_categorical_accuracy: 0.9608
Epoch 2/200
719/719 [==============================] - 2s 3ms/step - loss: 9.7398e-04 - sparse_categorical_accuracy: 1.0000
Epoch 3/200
719/719 [==============================] - 2s 3ms/step - loss: 3.1330e-04 - sparse_categorical_accuracy: 1.0000
Epoch 4/200
719/719 [==============================] - 2s 3ms/step - loss: 1.3280e-04 - sparse_categorical_accuracy: 1.0000
Epoch 5/200
719/719 [==============================] - 2s 3ms/step - loss: 6.4621e-05 - sparse_categorical_accuracy: 1.0000
Epoch 6/200
719/719 [==============================] - 2s 3ms/step - loss: 3.6452e-05 - sparse_categorical_accuracy: 1.0000
Epoch 7/200
719/719 [==============================] - 2s 3ms/step - loss: 2.1693e-05 - sparse_categorical_accuracy: 1.0000
Epoch 8/200
719/719 [==============================] - 2

In [0]:
Run('data/FNOW/MHEALTH.npz')

Proposed Model data/FNOW/MHEALTH.npz
12
(12015, 93) (12015,)
(1335, 93) (1335,)
Epoch 1/100
376/376 [==============================] - 1s 3ms/step - loss: 0.3589 - sparse_categorical_accuracy: 0.9134
Epoch 2/100
376/376 [==============================] - 1s 3ms/step - loss: 0.0440 - sparse_categorical_accuracy: 0.9858
Epoch 3/100
376/376 [==============================] - 1s 3ms/step - loss: 0.0217 - sparse_categorical_accuracy: 0.9928
Epoch 4/100
376/376 [==============================] - 1s 3ms/step - loss: 0.0086 - sparse_categorical_accuracy: 0.9967
Epoch 5/100
376/376 [==============================] - 1s 3ms/step - loss: 0.0030 - sparse_categorical_accuracy: 0.9993
Epoch 6/100
376/376 [==============================] - 1s 3ms/step - loss: 0.0395 - sparse_categorical_accuracy: 0.9903
Epoch 7/100
376/376 [==============================] - 1s 3ms/step - loss: 0.0019 - sparse_categorical_accuracy: 0.9996
Epoch 8/100
376/376 [==============================] - 1s 3ms/step - loss: 4.073

In [0]:
Run('data/SNOW/MHEALTH.npz')

Proposed Model data/SNOW/MHEALTH.npz
12
(22995, 93) (22995,)
(2555, 93) (2555,)
Epoch 1/100
719/719 [==============================] - 2s 3ms/step - loss: 0.3815 - sparse_categorical_accuracy: 0.9329
Epoch 2/100
719/719 [==============================] - 2s 3ms/step - loss: 0.0210 - sparse_categorical_accuracy: 0.9927
Epoch 3/100
719/719 [==============================] - 2s 3ms/step - loss: 0.0104 - sparse_categorical_accuracy: 0.9966
Epoch 4/100
719/719 [==============================] - 2s 3ms/step - loss: 0.0127 - sparse_categorical_accuracy: 0.9956
Epoch 5/100
719/719 [==============================] - 2s 3ms/step - loss: 0.0104 - sparse_categorical_accuracy: 0.9968
Epoch 6/100
719/719 [==============================] - 2s 3ms/step - loss: 0.0012 - sparse_categorical_accuracy: 0.9995
Epoch 7/100
719/719 [==============================] - 2s 3ms/step - loss: 0.0222 - sparse_categorical_accuracy: 0.9933
Epoch 8/100
719/719 [==============================] - 2s 3ms/step - loss: 5.682

In [0]:
Run('data/LOTO/USCHAD.npz')

Proposed Model data/LOTO/USCHAD.npz
12
(88416, 7) (88416,)
(9824, 7) (9824,)
Epoch 1/200
2763/2763 [==============================] - 7s 3ms/step - loss: 0.4066 - sparse_categorical_accuracy: 0.8428
Epoch 2/200
2763/2763 [==============================] - 7s 3ms/step - loss: 0.2401 - sparse_categorical_accuracy: 0.8960
Epoch 3/200
2763/2763 [==============================] - 7s 2ms/step - loss: 0.2013 - sparse_categorical_accuracy: 0.9083
Epoch 4/200
2763/2763 [==============================] - 7s 2ms/step - loss: 0.1762 - sparse_categorical_accuracy: 0.9179
Epoch 5/200
2763/2763 [==============================] - 7s 3ms/step - loss: 0.1611 - sparse_categorical_accuracy: 0.9218
Epoch 6/200
2763/2763 [==============================] - 7s 2ms/step - loss: 0.1501 - sparse_categorical_accuracy: 0.9268
Epoch 7/200
2763/2763 [==============================] - 7s 2ms/step - loss: 0.1391 - sparse_categorical_accuracy: 0.9302
Epoch 8/200
2763/2763 [==============================] - 7s 3ms/step 

In [0]:
Run('data/LOSO/USCHAD.npz')

Proposed Model data/LOSO/USCHAD.npz
12
(127712, 7) (127712,)
(9824, 7) (9824,)
Epoch 1/200
3991/3991 [==============================] - 10s 2ms/step - loss: 0.3660 - sparse_categorical_accuracy: 0.8538
Epoch 2/200
3991/3991 [==============================] - 10s 2ms/step - loss: 0.2018 - sparse_categorical_accuracy: 0.9091
Epoch 3/200
3991/3991 [==============================] - 10s 2ms/step - loss: 0.1656 - sparse_categorical_accuracy: 0.9216
Epoch 4/200
3991/3991 [==============================] - 10s 2ms/step - loss: 0.1470 - sparse_categorical_accuracy: 0.9283
Epoch 5/200
3991/3991 [==============================] - 10s 2ms/step - loss: 0.1350 - sparse_categorical_accuracy: 0.9324
Epoch 6/200
3991/3991 [==============================] - 10s 2ms/step - loss: 0.1283 - sparse_categorical_accuracy: 0.9352
Epoch 7/200
3991/3991 [==============================] - 10s 2ms/step - loss: 0.1201 - sparse_categorical_accuracy: 0.9385
Epoch 8/200
3991/3991 [==============================] - 10s

In [0]:
Run('data/FNOW/USCHAD.npz')

Proposed Model data/FNOW/USCHAD.npz
12
(45342, 7) (45342,)
(5038, 7) (5038,)
Epoch 1/200
1417/1417 [==============================] - 4s 3ms/step - loss: 0.5453 - sparse_categorical_accuracy: 0.7945
Epoch 2/200
1417/1417 [==============================] - 4s 3ms/step - loss: 0.3056 - sparse_categorical_accuracy: 0.8748
Epoch 3/200
1417/1417 [==============================] - 4s 3ms/step - loss: 0.2486 - sparse_categorical_accuracy: 0.8938
Epoch 4/200
1417/1417 [==============================] - 4s 3ms/step - loss: 0.2130 - sparse_categorical_accuracy: 0.9078
Epoch 5/200
1417/1417 [==============================] - 4s 3ms/step - loss: 0.1932 - sparse_categorical_accuracy: 0.9134
Epoch 6/200
1417/1417 [==============================] - 4s 3ms/step - loss: 0.1751 - sparse_categorical_accuracy: 0.9204
Epoch 7/200
1417/1417 [==============================] - 4s 3ms/step - loss: 0.1629 - sparse_categorical_accuracy: 0.9229
Epoch 8/200
1417/1417 [==============================] - 4s 3ms/step 

In [0]:
Run('data/SNOW/USCHAD.npz')

Proposed Model data/SNOW/USCHAD.npz
12
(88416, 25) (88416,)
(9824, 25) (9824,)
Epoch 1/200
2763/2763 [==============================] - 7s 3ms/step - loss: 0.4899 - sparse_categorical_accuracy: 0.8120
Epoch 2/200
2763/2763 [==============================] - 7s 2ms/step - loss: 0.2414 - sparse_categorical_accuracy: 0.8947
Epoch 3/200
2763/2763 [==============================] - 7s 2ms/step - loss: 0.1946 - sparse_categorical_accuracy: 0.9135
Epoch 4/200
2763/2763 [==============================] - 7s 2ms/step - loss: 0.1628 - sparse_categorical_accuracy: 0.9236
Epoch 5/200
2763/2763 [==============================] - 7s 2ms/step - loss: 0.1495 - sparse_categorical_accuracy: 0.9297
Epoch 6/200
2763/2763 [==============================] - 7s 2ms/step - loss: 0.1402 - sparse_categorical_accuracy: 0.9347
Epoch 7/200
2763/2763 [==============================] - 7s 2ms/step - loss: 0.1292 - sparse_categorical_accuracy: 0.9378
Epoch 8/200
2763/2763 [==============================] - 7s 2ms/ste

In [0]:
Run('data/LOTO/WHARF.npz')

Proposed Model data/LOTO/WHARF.npz
12
(34839, 5) (34839,)
(3871, 5) (3871,)
Epoch 1/200
1089/1089 [==============================] - 3s 3ms/step - loss: 1.0019 - sparse_categorical_accuracy: 0.6307
Epoch 2/200
1089/1089 [==============================] - 3s 3ms/step - loss: 0.7830 - sparse_categorical_accuracy: 0.6986
Epoch 3/200
1089/1089 [==============================] - 3s 3ms/step - loss: 0.7185 - sparse_categorical_accuracy: 0.7216
Epoch 4/200
1089/1089 [==============================] - 3s 3ms/step - loss: 0.6761 - sparse_categorical_accuracy: 0.7390
Epoch 5/200
1089/1089 [==============================] - 3s 3ms/step - loss: 0.6486 - sparse_categorical_accuracy: 0.7479
Epoch 6/200
1089/1089 [==============================] - 3s 3ms/step - loss: 0.6189 - sparse_categorical_accuracy: 0.7597
Epoch 7/200
1089/1089 [==============================] - 3s 3ms/step - loss: 0.5937 - sparse_categorical_accuracy: 0.7683
Epoch 8/200
1089/1089 [==============================] - 3s 3ms/step -

In [0]:
Run('data/LOSO/WHARF.npz')

Proposed Model data/LOSO/WHARF.npz
12
(58065, 5) (58065,)
(3871, 5) (3871,)
Epoch 1/200
1815/1815 [==============================] - 5s 3ms/step - loss: 0.9245 - sparse_categorical_accuracy: 0.6541
Epoch 2/200
1815/1815 [==============================] - 5s 3ms/step - loss: 0.7166 - sparse_categorical_accuracy: 0.7190
Epoch 3/200
1815/1815 [==============================] - 5s 3ms/step - loss: 0.6457 - sparse_categorical_accuracy: 0.7451
Epoch 4/200
1815/1815 [==============================] - 5s 3ms/step - loss: 0.5908 - sparse_categorical_accuracy: 0.7671
Epoch 5/200
1815/1815 [==============================] - 5s 3ms/step - loss: 0.5518 - sparse_categorical_accuracy: 0.7838
Epoch 6/200
1815/1815 [==============================] - 5s 3ms/step - loss: 0.5131 - sparse_categorical_accuracy: 0.7975
Epoch 7/200
1815/1815 [==============================] - 5s 3ms/step - loss: 0.4807 - sparse_categorical_accuracy: 0.8111
Epoch 8/200
1815/1815 [==============================] - 5s 3ms/step -

In [0]:
Run('data/SNOW/WHARF.npz')

Proposed Model data/SNOW/WHARF.npz
12
(34920, 5) (34920,)
(3880, 5) (3880,)
Epoch 1/200
1092/1092 [==============================] - 3s 3ms/step - loss: 0.9969 - sparse_categorical_accuracy: 0.6346
Epoch 2/200
1092/1092 [==============================] - 3s 3ms/step - loss: 0.7818 - sparse_categorical_accuracy: 0.6978
Epoch 3/200
1092/1092 [==============================] - 3s 3ms/step - loss: 0.7169 - sparse_categorical_accuracy: 0.7214
Epoch 4/200
1092/1092 [==============================] - 3s 3ms/step - loss: 0.6757 - sparse_categorical_accuracy: 0.7358
Epoch 5/200
1092/1092 [==============================] - 3s 3ms/step - loss: 0.6442 - sparse_categorical_accuracy: 0.7491
Epoch 6/200
1092/1092 [==============================] - 3s 3ms/step - loss: 0.6219 - sparse_categorical_accuracy: 0.7571
Epoch 7/200
1092/1092 [==============================] - 3s 3ms/step - loss: 0.5936 - sparse_categorical_accuracy: 0.7673
Epoch 8/200
1092/1092 [==============================] - 3s 3ms/step -

In [0]:
Run('data/FNOW/WHARF.npz')

Proposed Model data/FNOW/WHARF.npz
12
(19314, 5) (19314,)
(2146, 5) (2146,)
Epoch 1/200
604/604 [==============================] - 2s 3ms/step - loss: 1.1427 - sparse_categorical_accuracy: 0.5854
Epoch 2/200
604/604 [==============================] - 2s 3ms/step - loss: 0.8696 - sparse_categorical_accuracy: 0.6615
Epoch 3/200
604/604 [==============================] - 2s 3ms/step - loss: 0.8029 - sparse_categorical_accuracy: 0.6835
Epoch 4/200
604/604 [==============================] - 2s 3ms/step - loss: 0.7549 - sparse_categorical_accuracy: 0.6991
Epoch 5/200
604/604 [==============================] - 2s 3ms/step - loss: 0.7172 - sparse_categorical_accuracy: 0.7136
Epoch 6/200
604/604 [==============================] - 2s 3ms/step - loss: 0.6839 - sparse_categorical_accuracy: 0.7289
Epoch 7/200
604/604 [==============================] - 2s 3ms/step - loss: 0.6562 - sparse_categorical_accuracy: 0.7372
Epoch 8/200
604/604 [==============================] - 2s 3ms/step - loss: 0.6320 - 

In [0]:
Run('data/LOTO/WISDM.npz')

Proposed Model data/LOTO/WISDM.npz
6
(187614, 5) (187614,)
(20846, 5) (20846,)
Epoch 1/200
5863/5863 [==============================] - 17s 3ms/step - loss: 0.3047 - sparse_categorical_accuracy: 0.8899
Epoch 2/200
5863/5863 [==============================] - 17s 3ms/step - loss: 0.2038 - sparse_categorical_accuracy: 0.9256
Epoch 3/200
5863/5863 [==============================] - 17s 3ms/step - loss: 0.1734 - sparse_categorical_accuracy: 0.9371
Epoch 4/200
5863/5863 [==============================] - 17s 3ms/step - loss: 0.1529 - sparse_categorical_accuracy: 0.9455
Epoch 5/200
5863/5863 [==============================] - 17s 3ms/step - loss: 0.1384 - sparse_categorical_accuracy: 0.9511
Epoch 6/200
5863/5863 [==============================] - 17s 3ms/step - loss: 0.1275 - sparse_categorical_accuracy: 0.9550
Epoch 7/200
5863/5863 [==============================] - 17s 3ms/step - loss: 0.1176 - sparse_categorical_accuracy: 0.9583
Epoch 8/200
5863/5863 [==============================] - 17s

In [0]:
Run('data/LOSO/WISDM.npz')

Proposed Model data/LOSO/WISDM.npz
6
(729610, 13) (729610,)
(20846, 13) (20846,)
Epoch 1/200
22801/22801 [==============================] - 62s 3ms/step - loss: 0.1958 - sparse_categorical_accuracy: 0.9296
Epoch 2/200
22801/22801 [==============================] - 62s 3ms/step - loss: 0.0896 - sparse_categorical_accuracy: 0.9684
Epoch 3/200
22801/22801 [==============================] - 62s 3ms/step - loss: 0.0668 - sparse_categorical_accuracy: 0.9764
Epoch 4/200
22801/22801 [==============================] - 61s 3ms/step - loss: 0.0544 - sparse_categorical_accuracy: 0.9810
Epoch 5/200
22801/22801 [==============================] - 63s 3ms/step - loss: 0.0463 - sparse_categorical_accuracy: 0.9836
Epoch 6/200
22801/22801 [==============================] - 61s 3ms/step - loss: 0.0400 - sparse_categorical_accuracy: 0.9861
Epoch 7/200
22801/22801 [==============================] - 62s 3ms/step - loss: 0.0356 - sparse_categorical_accuracy: 0.9877
Epoch 8/200
22801/22801 [===================

In [0]:
Run('data/SNOW/WISDM.npz')

Proposed Model data/SNOW/WISDM.npz
6
(187614, 13) (187614,)
(20846, 13) (20846,)
Epoch 1/200
5863/5863 [==============================] - 16s 3ms/step - loss: 0.3288 - sparse_categorical_accuracy: 0.8825
Epoch 2/200
5863/5863 [==============================] - 17s 3ms/step - loss: 0.1855 - sparse_categorical_accuracy: 0.9328
Epoch 3/200
5863/5863 [==============================] - 17s 3ms/step - loss: 0.1407 - sparse_categorical_accuracy: 0.9501
Epoch 4/200
5863/5863 [==============================] - 16s 3ms/step - loss: 0.1173 - sparse_categorical_accuracy: 0.9585
Epoch 5/200
5863/5863 [==============================] - 16s 3ms/step - loss: 0.1035 - sparse_categorical_accuracy: 0.9637
Epoch 6/200
5863/5863 [==============================] - 16s 3ms/step - loss: 0.0930 - sparse_categorical_accuracy: 0.9670
Epoch 7/200
5863/5863 [==============================] - 16s 3ms/step - loss: 0.0833 - sparse_categorical_accuracy: 0.9710
Epoch 8/200
5863/5863 [==============================] - 1

In [0]:
Run('data/FNOW/WISDM.npz')

Proposed Model data/FNOW/WISDM.npz
6
(94644, 13) (94644,)
(10516, 13) (10516,)
Epoch 1/200
2958/2958 [==============================] - 8s 3ms/step - loss: 0.4187 - sparse_categorical_accuracy: 0.8504
Epoch 2/200
2958/2958 [==============================] - 8s 3ms/step - loss: 0.2870 - sparse_categorical_accuracy: 0.8945
Epoch 3/200
2958/2958 [==============================] - 8s 3ms/step - loss: 0.2249 - sparse_categorical_accuracy: 0.9173
Epoch 4/200
2958/2958 [==============================] - 8s 3ms/step - loss: 0.1841 - sparse_categorical_accuracy: 0.9331
Epoch 5/200
2958/2958 [==============================] - 8s 3ms/step - loss: 0.1588 - sparse_categorical_accuracy: 0.9438
Epoch 6/200
2958/2958 [==============================] - 8s 3ms/step - loss: 0.1438 - sparse_categorical_accuracy: 0.9500
Epoch 7/200
2958/2958 [==============================] - 8s 3ms/step - loss: 0.1306 - sparse_categorical_accuracy: 0.9545
Epoch 8/200
2958/2958 [==============================] - 8s 3ms/ste

In [0]:
Run('data/LOTO/UTD-MHAD2_1s.npz')

Proposed Model data/LOTO/UTD-MHAD2_1s.npz
6
(10233, 8) (10233,)
(1137, 8) (1137,)
Epoch 1/200
320/320 [==============================] - 1s 3ms/step - loss: 0.7244 - sparse_categorical_accuracy: 0.7239
Epoch 2/200
320/320 [==============================] - 1s 3ms/step - loss: 0.4211 - sparse_categorical_accuracy: 0.8341
Epoch 3/200
320/320 [==============================] - 1s 3ms/step - loss: 0.3278 - sparse_categorical_accuracy: 0.8706
Epoch 4/200
320/320 [==============================] - 1s 3ms/step - loss: 0.2769 - sparse_categorical_accuracy: 0.8936
Epoch 5/200
320/320 [==============================] - 1s 3ms/step - loss: 0.2387 - sparse_categorical_accuracy: 0.9129
Epoch 6/200
320/320 [==============================] - 1s 3ms/step - loss: 0.2139 - sparse_categorical_accuracy: 0.9210
Epoch 7/200
320/320 [==============================] - 1s 3ms/step - loss: 0.1872 - sparse_categorical_accuracy: 0.9334
Epoch 8/200
320/320 [==============================] - 1s 3ms/step - loss: 0.1

In [0]:
Run('data/LOSO/UTD-MHAD2_1s.npz')

Proposed Model data/LOSO/UTD-MHAD2_1s.npz
6
(7959, 8) (7959,)
(1137, 8) (1137,)
Epoch 1/200
249/249 [==============================] - 1s 3ms/step - loss: 0.7654 - sparse_categorical_accuracy: 0.7007
Epoch 2/200
249/249 [==============================] - 1s 3ms/step - loss: 0.4443 - sparse_categorical_accuracy: 0.8246
Epoch 3/200
249/249 [==============================] - 1s 3ms/step - loss: 0.3561 - sparse_categorical_accuracy: 0.8610
Epoch 4/200
249/249 [==============================] - 1s 3ms/step - loss: 0.3041 - sparse_categorical_accuracy: 0.8847
Epoch 5/200
249/249 [==============================] - 1s 3ms/step - loss: 0.2674 - sparse_categorical_accuracy: 0.8979
Epoch 6/200
249/249 [==============================] - 1s 3ms/step - loss: 0.2427 - sparse_categorical_accuracy: 0.9064
Epoch 7/200
249/249 [==============================] - 1s 3ms/step - loss: 0.2138 - sparse_categorical_accuracy: 0.9182
Epoch 8/200
249/249 [==============================] - 1s 3ms/step - loss: 0.192

In [0]:
Run('data/FNOW/UTD-MHAD2_1s.npz')

Proposed Model data/FNOW/UTD-MHAD2_1s.npz
6
(5544, 8) (5544,)
(616, 8) (616,)
Epoch 1/200
174/174 [==============================] - 1s 3ms/step - loss: 0.8817 - sparse_categorical_accuracy: 0.6611
Epoch 2/200
174/174 [==============================] - 1s 3ms/step - loss: 0.4480 - sparse_categorical_accuracy: 0.8348
Epoch 3/200
174/174 [==============================] - 0s 3ms/step - loss: 0.3364 - sparse_categorical_accuracy: 0.8804
Epoch 4/200
174/174 [==============================] - 0s 3ms/step - loss: 0.2711 - sparse_categorical_accuracy: 0.9062
Epoch 5/200
174/174 [==============================] - 1s 3ms/step - loss: 0.2252 - sparse_categorical_accuracy: 0.9194
Epoch 6/200
174/174 [==============================] - 0s 3ms/step - loss: 0.1934 - sparse_categorical_accuracy: 0.9316
Epoch 7/200
174/174 [==============================] - 0s 3ms/step - loss: 0.1535 - sparse_categorical_accuracy: 0.9499
Epoch 8/200
174/174 [==============================] - 1s 3ms/step - loss: 0.1338 

In [0]:
Run('data/SNOW/UTD-MHAD2_1s.npz')

Proposed Model data/SNOW/UTD-MHAD2_1s.npz
6
(10233, 8) (10233,)
(1137, 8) (1137,)
Epoch 1/200
320/320 [==============================] - 1s 3ms/step - loss: 0.7327 - sparse_categorical_accuracy: 0.7315
Epoch 2/200
320/320 [==============================] - 1s 3ms/step - loss: 0.4239 - sparse_categorical_accuracy: 0.8362
Epoch 3/200
320/320 [==============================] - 1s 3ms/step - loss: 0.3328 - sparse_categorical_accuracy: 0.8690
Epoch 4/200
320/320 [==============================] - 1s 3ms/step - loss: 0.2773 - sparse_categorical_accuracy: 0.8951
Epoch 5/200
320/320 [==============================] - 1s 3ms/step - loss: 0.2315 - sparse_categorical_accuracy: 0.9130
Epoch 6/200
320/320 [==============================] - 1s 3ms/step - loss: 0.2048 - sparse_categorical_accuracy: 0.9242
Epoch 7/200
320/320 [==============================] - 1s 3ms/step - loss: 0.1740 - sparse_categorical_accuracy: 0.9361
Epoch 8/200
320/320 [==============================] - 1s 3ms/step - loss: 0.1

In [0]:
Run('data/LOTO/UTD-MHAD1_1s.npz')

Proposed Model data/LOTO/UTD-MHAD1_1s.npz
21
(33939, 10) (33939,)
(3771, 10) (3771,)
Epoch 1/200
1061/1061 [==============================] - 3s 3ms/step - loss: 1.5411 - sparse_categorical_accuracy: 0.5142
Epoch 2/200
1061/1061 [==============================] - 3s 3ms/step - loss: 0.9741 - sparse_categorical_accuracy: 0.6768
Epoch 3/200
1061/1061 [==============================] - 3s 3ms/step - loss: 0.7821 - sparse_categorical_accuracy: 0.7429
Epoch 4/200
1061/1061 [==============================] - 3s 3ms/step - loss: 0.6538 - sparse_categorical_accuracy: 0.7864
Epoch 5/200
1061/1061 [==============================] - 3s 3ms/step - loss: 0.5575 - sparse_categorical_accuracy: 0.8200
Epoch 6/200
1061/1061 [==============================] - 3s 3ms/step - loss: 0.4752 - sparse_categorical_accuracy: 0.8488
Epoch 7/200
1061/1061 [==============================] - 3s 3ms/step - loss: 0.4064 - sparse_categorical_accuracy: 0.8737
Epoch 8/200
1061/1061 [==============================] - 3s 3

In [0]:
Run('data/LOSO/UTD-MHAD1_1s.npz')

Proposed Model data/LOSO/UTD-MHAD1_1s.npz
21
(26397, 10) (26397,)
(3771, 10) (3771,)
Epoch 1/200
825/825 [==============================] - 2s 3ms/step - loss: 1.6543 - sparse_categorical_accuracy: 0.4762
Epoch 2/200
825/825 [==============================] - 2s 3ms/step - loss: 1.0413 - sparse_categorical_accuracy: 0.6592
Epoch 3/200
825/825 [==============================] - 2s 3ms/step - loss: 0.8357 - sparse_categorical_accuracy: 0.7306
Epoch 4/200
825/825 [==============================] - 2s 3ms/step - loss: 0.7065 - sparse_categorical_accuracy: 0.7736
Epoch 5/200
825/825 [==============================] - 2s 3ms/step - loss: 0.6103 - sparse_categorical_accuracy: 0.8065
Epoch 6/200
825/825 [==============================] - 3s 3ms/step - loss: 0.5275 - sparse_categorical_accuracy: 0.8333
Epoch 7/200
825/825 [==============================] - 2s 3ms/step - loss: 0.4672 - sparse_categorical_accuracy: 0.8551
Epoch 8/200
825/825 [==============================] - 2s 3ms/step - loss: 

In [0]:
Run('data/SNOW/UTD-MHAD1_1s.npz')

Proposed Model data/SNOW/UTD-MHAD1_1s.npz
21
(33939, 10) (33939,)
(3771, 10) (3771,)
Epoch 1/200
1061/1061 [==============================] - 3s 3ms/step - loss: 1.5632 - sparse_categorical_accuracy: 0.4993
Epoch 2/200
1061/1061 [==============================] - 3s 3ms/step - loss: 0.9823 - sparse_categorical_accuracy: 0.6734
Epoch 3/200
1061/1061 [==============================] - 3s 3ms/step - loss: 0.7829 - sparse_categorical_accuracy: 0.7419
Epoch 4/200
1061/1061 [==============================] - 3s 3ms/step - loss: 0.6546 - sparse_categorical_accuracy: 0.7882
Epoch 5/200
1061/1061 [==============================] - 3s 3ms/step - loss: 0.5569 - sparse_categorical_accuracy: 0.8227
Epoch 6/200
1061/1061 [==============================] - 3s 3ms/step - loss: 0.4745 - sparse_categorical_accuracy: 0.8505
Epoch 7/200
1061/1061 [==============================] - 3s 3ms/step - loss: 0.4052 - sparse_categorical_accuracy: 0.8746
Epoch 8/200
1061/1061 [==============================] - 3s 3

In [0]:
Run('data/FNOW/UTD-MHAD1_1s.npz')

Proposed Model data/FNOW/UTD-MHAD1_1s.npz
21
(18432, 10) (18432,)
(2048, 10) (2048,)
Epoch 1/200
576/576 [==============================] - 2s 3ms/step - loss: 1.7426 - sparse_categorical_accuracy: 0.4562
Epoch 2/200
576/576 [==============================] - 1s 3ms/step - loss: 0.9963 - sparse_categorical_accuracy: 0.6816
Epoch 3/200
576/576 [==============================] - 2s 3ms/step - loss: 0.7432 - sparse_categorical_accuracy: 0.7704
Epoch 4/200
576/576 [==============================] - 2s 3ms/step - loss: 0.5819 - sparse_categorical_accuracy: 0.8291
Epoch 5/200
576/576 [==============================] - 2s 3ms/step - loss: 0.4768 - sparse_categorical_accuracy: 0.8615
Epoch 6/200
576/576 [==============================] - 2s 3ms/step - loss: 0.3866 - sparse_categorical_accuracy: 0.8919
Epoch 7/200
576/576 [==============================] - 2s 3ms/step - loss: 0.3147 - sparse_categorical_accuracy: 0.9144
Epoch 8/200
576/576 [==============================] - 2s 3ms/step - loss: 